### CREATING NEW AGENT FLOW


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

# This will be a tool
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b

def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b



#llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

In [ ]:
from typing_extensions import TypedDict
from typing_extensions import TypedDict
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage, ToolMessage, AnyMessage
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode

class CreatorGraphState(TypedDict):
    # Place Variables Here
    
    full_results: SystemMessage           # Last results of running the game
    analysis: AIMessage         # Output of Anlayzer, What Happend?
    solution: AIMessage         # Ouput of Researcher, What should be done?
    new_code: AIMessage         # Output of Coder
    test_results: SystemMessage           # Running a test on code, to ensure correctness
    validation: AIMessage        # Ouptut of Validator, Is the code correct?

    evolve_counter: int         # Counter for the number of evolutions


#tools = [add, multiply, divide]
DEFAULT_EVOLVE_COUNTER = 3
#llm = ChatOpenAI(model="gpt-4o")
llm = AzureChatOpenAI(
    model="gpt-4o",
    azure_endpoint="https://gpt-amayuelas.openai.azure.com/",
    api_version = "2024-12-01-preview"
)

def tool_calling_state_graph(sys_msg: SystemMessage, msgs: list[AnyMessage], tools):
    # Node

    # Bind Tools to the LLM
    llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)

    def assistant(state: MessagesState):
        return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}
    
    # Graph
    builder = StateGraph(MessagesState)

    # Define nodes: these do the work
    builder.add_node("assistant", assistant)
    builder.add_node("tools", ToolNode(tools))

    # Define edges: these determine how the control flow moves
    builder.add_edge(START, "assistant")
    builder.add_conditional_edges(
        "assistant",
        # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
        # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
        tools_condition,
    )
    builder.add_edge("tools", "assistant")
    react_graph = builder.compile()
    
    
    messages = react_graph.invoke({"messages": msgs})
    
    for m in messages['messages']:
        m.pretty_print()

    return messages


def run_player_node(state: CreatorGraphState):
    """
    Runs Catanatron with the current Code
    """
    #print("In Run Player Node")

    # Generate a test results (later will be running the game)
    output = llm.invoke([HumanMessage(content="Create me a complicated algebra math problem, without a solution. Return only the problem...NO COMMENTARY!!")])
    
    # Clear all past messages
    return {
        "full_results": SystemMessage(content=output.content),
        "analysis": AIMessage(content=""),
        "solution": AIMessage(content=""),
        "new_code": AIMessage(content=""),
        "test_results": SystemMessage(content=""),
        "validation": AIMessage(content="")
    }


def test_player_node(state: CreatorGraphState):
    """
    Tests Catanatron with the current Code
    """
    #print("In Test Player Node")

    return {"test_results": SystemMessage(content="These are the test results: YOU PASSED THE TEST!")}

def analyzer_node(state: CreatorGraphState):
    #print("In Analyzer Node")

    # If evolve_counter isnt initialized, set it to 0. If it is, increment it
    if "evolve_counter" not in state:
        evolve_counter = DEFAULT_EVOLVE_COUNTER
    else:
        evolve_counter = state["evolve_counter"] - 1


    sys_msg = SystemMessage(
        content="""
            You are a Math Agent. I want you to output just the analysis of the problem, not the solution "
            Make sure to start your output with 'ANALYSIS:' and end with 'END ANALYSIS'. No Commentary, just the solution.
        """
    )
    msg = [state["full_results"]]
    tools = []
    output = tool_calling_state_graph(sys_msg, msg, tools)

    #print(output)
    return {"analysis": output["messages"][-1], "evolve_counter": evolve_counter}
 
def researcher_node(state: CreatorGraphState):
    
    #print("In Researcher Node")
    # Add custom tools for researcher

    sys_msg = SystemMessage(
        content="""
            You are a Math Agent. I want you to research the given analysis on the previous equation, and research a way to achieve the solution.
            This does not need specifically be the solution. Make sure to start your output with 'SOLUTION:' and end with 'END SOLUTION'. No Commentary, just the solution.
        """
    )
    msg = [state["analysis"], state["full_results"]]
    tools = []
    output = tool_calling_state_graph(sys_msg, msg, tools)
    return {"solution": output["messages"][-1]}

def coder_node(state: CreatorGraphState):

    #print("In Researcher Node")
    # Add custom tools for researcher

    sys_msg = SystemMessage(
        content="""
            You are a Math Agent. I want you take in the proposed solution, and implement it to output the result
        """
    )
    msg = [state["solution"]]
    tools = []
    output = tool_calling_state_graph(sys_msg, msg, tools)
    return {"new_code": output["messages"][-1]}

val_ok_key = "PASSSED_VALIDATION"
val_not_ok_key = "FAILED_VALIDATION"

def validator_node(state: CreatorGraphState):
    """
    Validates the code
    """
    #print("In Validator Node")
    # Add Custom Tools For Validator
    
    sys_msg = SystemMessage(
        content=f"""
            You are a Math Agent. The previous system message is an external validation tool as a results of your solution, which is in the previous message.
            I want you to look at the results of the test, and follow the following instructions:
            If the results are correct, return the key "{val_ok_key}", if the results are correct, return the key "{val_not_ok_key}".
            Then, respond with an analysis and explanation of what key you decided to return.
             Make sure to start your output with 'VALIDATION:' and end with 'END VALIDATION'. No Commentary, just the solution.
        """
    )
    msg = [state["new_code"], state["test_results"], ]
    tools = []
    output = tool_calling_state_graph(sys_msg, msg, tools)

    return {"validation": output["messages"][-1]}

def conditional_edge_validator(state: CreatorGraphState):
    """
    Conditional edge for validator
    """
    print("In Conditional Edge Validator")
    
    # Get the content of the validation message
    validation_message = state["validation"].content
    
    # Check for the presence of our defined result strings
    if state["evolve_counter"] <= 0:
        print("Evolve counter is 0 - ending workflow")
        return END

    if val_ok_key in validation_message:
        print("Validation passed - ending workflow")
        return END
    elif val_not_ok_key in validation_message:  #
        print("Validation failed - rerunning player")
        return "run_player"
    else:
        # Default case if neither string is found
        print("Warning: Could not determine validation result, defaulting to END")
        return END


graph = StateGraph(CreatorGraphState)

graph.add_node("run_player", run_player_node)
graph.add_node("analyzer", analyzer_node)
#graph.add_node("tools", ToolNode(tools))
graph.add_node("researcher", researcher_node)
graph.add_node("coder", coder_node)
graph.add_node("test_player", test_player_node)
graph.add_node("validator", validator_node)

graph.add_edge(START, "run_player")
graph.add_edge("run_player", "analyzer")
graph.add_edge("analyzer", "researcher")
graph.add_edge("researcher", "coder")
graph.add_edge("coder", "test_player")
graph.add_edge("test_player", "validator")
graph.add_conditional_edges(
    "validator",
    conditional_edge_validator,
    {END, "run_player"}
)
#graph.add_edge("validator", END)

graph = graph.compile()





In [77]:
#display(Image(graph.get_graph().draw_mermaid_png(max_retries=5, retry_delay=2.0)))


In [129]:


initial_state: CreatorGraphState = {}
for step in graph.stream(initial_state, stream_mode="updates"):
    print(step)
    #print(event)
    # if event["thinking_node"] :
    #     print(event["thinking_node"]["solution"][0].content)
    # elif event["answer_node"]:
    #     print(event["answer_node"]["formated"][0].content)
    
#messages = graph.invoke({"problem": msg})


# for m in messages['messages']:
#     m.pretty_print()

{'run_player': {'full_results': SystemMessage(content="Sure, here is the problem:\n\nIf \\( f(x) = \\frac{3x^2 - 5x + 7}{x^2 - 4x + 4} \\), \\( g(x) = 2x^3 - x^2 + 3x - 6 \\), and \\( h(x) = e^{x^2} + \\ln(x+3) \\), solve for \\( x \\) such that:\n\n\\[\n\\int_{1}^{2} \\left[ f(x) \\cdot g'(x) + h''(x) \\right] \\, dx = \\sum_{n=1}^{\\infty} \\frac{(-1)^n}{n^2 + n} + \\lim_{x \\to 2^-} \\frac{g(x)}{h(x)} \\cdot \\frac{d}{dx}\\left(f(x) \\cdot h(x)\\right)\n\\]", additional_kwargs={}, response_metadata={}), 'analysis': AIMessage(content='', additional_kwargs={}, response_metadata={}), 'solution': AIMessage(content='', additional_kwargs={}, response_metadata={}), 'new_code': AIMessage(content='', additional_kwargs={}, response_metadata={}), 'test_results': SystemMessage(content='', additional_kwargs={}, response_metadata={}), 'validation': AIMessage(content='', additional_kwargs={}, response_metadata={})}}
================================ System Message ================================



KeyboardInterrupt: 